In [1]:
import os
import sys
import random
import time
import argparse
import numpy as np
import matplotlib.pyplot as plt

import multiprocessing as mp

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

from models import DPINet
#from data import PhysicsFleXDataset, collate_fn

from utils import count_parameters

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('--pstep', type=int, default=2)
parser.add_argument('--n_rollout', type=int, default=0)
parser.add_argument('--time_step', type=int, default=0)
parser.add_argument('--time_step_clip', type=int, default=0)
parser.add_argument('--dt', type=float, default=1./60.)
parser.add_argument('--nf_relation', type=int, default=300)
parser.add_argument('--nf_particle', type=int, default=200)
parser.add_argument('--nf_effect', type=int, default=200)
parser.add_argument('--env', default='')
parser.add_argument('--train_valid_ratio', type=float, default=0.9)
parser.add_argument('--outf', default='files')
parser.add_argument('--dataf', default='data')
parser.add_argument('--num_workers', type=int, default=10)
parser.add_argument('--gen_data', type=int, default=0)
parser.add_argument('--gen_stat', type=int, default=0)
parser.add_argument('--log_per_iter', type=int, default=1000)
parser.add_argument('--ckp_per_iter', type=int, default=10000)
parser.add_argument('--eval', type=int, default=0)
parser.add_argument('--verbose_data', type=int, default=1)
parser.add_argument('--verbose_model', type=int, default=1)

parser.add_argument('--n_instance', type=int, default=0)
parser.add_argument('--n_stages', type=int, default=0)
parser.add_argument('--n_his', type=int, default=0)

parser.add_argument('--n_epoch', type=int, default=1000)
parser.add_argument('--beta1', type=float, default=0.9)
parser.add_argument('--lr', type=float, default=0.0001)
parser.add_argument('--batch_size', type=int, default=1)
parser.add_argument('--forward_times', type=int, default=2)

parser.add_argument('--resume_epoch', type=int, default=0)
parser.add_argument('--resume_iter', type=int, default=0)

# shape state:
# [x, y, z, x_last, y_last, z_last, quat(4), quat_last(4)]
parser.add_argument('--shape_state_dim', type=int, default=14)

# object attributes:
parser.add_argument('--attr_dim', type=int, default=0)

# object state:
parser.add_argument('--state_dim', type=int, default=0)
parser.add_argument('--position_dim', type=int, default=0)

# relation attr:
parser.add_argument('--relation_dim', type=int, default=0)

_StoreAction(option_strings=['--relation_dim'], dest='relation_dim', nargs=None, const=None, default=0, type=<class 'int'>, choices=None, help=None, metavar=None)

In [3]:
args = parser.parse_args("--env SingleHair --gen_data 1".split())

In [4]:
phases_dict = dict()

args.n_rollout = 50
args.num_workers = 3
args.gen_stat = 1

# object states:
# [x, y, z, xdot, ydot, zdot]
args.state_dim = 6
args.position_dim = 3

# object attr:
# [rigid]
args.attr_dim = 1

# relation attr:
# [none]
args.relation_dim = 1

args.time_step = 500
args.time_step_clip = 20
args.n_instance = 1
args.n_stages = 1

args.neighbor_radius = 0.08

phases_dict["instance_idx"] = [0, 30]
phases_dict["root_num"] = [[]]
phases_dict["instance"] = ['solid']
phases_dict["material"] = ['solid']

args.outf = 'dump_SingleHair/' + args.outf

In [5]:
args.outf = args.outf + '_' + args.env
args.dataf = 'data/' + args.dataf + '_' + args.env
print (args.dataf)
os.system('mkdir -p ' + args.outf)
os.system('mkdir -p ' + args.dataf)

data/data_SingleHair


0

In [6]:
from data import *

In [7]:
def gen_PyFleX(info):

    env, root_num = info['env'], info['root_num']
    thread_idx, data_dir, data_names = info['thread_idx'], info['data_dir'], info['data_names']
    n_rollout, n_instance = info['n_rollout'], info['n_instance']
    time_step, time_step_clip = info['time_step'], info['time_step_clip']
    shape_state_dim, dt = info['shape_state_dim'], info['dt']

    env_idx = info['env_idx'] # =11

    np.random.seed(round(time.time() * 1000 + thread_idx) % 2**32)
    
    stats = [init_stat(3), init_stat(3)]

    import pyflex
    pyflex.init()

    for i in range(n_rollout):

        if i % 10 == 0:
            print("%d / %d" % (i, n_rollout))

        rollout_idx = thread_idx * n_rollout + i
        rollout_dir = os.path.join(data_dir, str(rollout_idx))
        os.system('mkdir -p ' + rollout_dir)
        
        # scene_params: [len(box) at dim x,len(box) at dim y,len(box) at dim z, num_hair per circle, num_circle]
        cap_size = [0.1,1.5]
        N_hairs = 1


        scene_params = np.array(cap_size)

        pyflex.set_scene(env_idx, scene_params, thread_idx)
        n_particles = pyflex.get_n_particles()
        n_shapes = 1
        N_particles_per_hair = int(n_particles/N_hairs)
        idx_begins = np.arange(N_hairs)*N_particles_per_hair
        idx_hairs = [[i,i+N_particles_per_hair-1] for i in idx_begins]

        positions = np.zeros((time_step, n_particles+ n_shapes, 3), dtype=np.float32)
        velocities = np.zeros((time_step, n_particles+ n_shapes, 3), dtype=np.float32)
   #     shape_position = np.zeros((time_step, n_shapes, 3), dtype=np.float32)
    #    shape_velocities = np.zeros((time_step, n_shapes, 3), dtype=np.float32)

        for j in range(time_step_clip):
         #   p_clip = pyflex.get_positions().reshape(-1, 4)[:, :3]
         #   shape_p_clip = pyflex.get_shape_states()[:3].reshape(-1,3)
            p_clip = np.concatenate([pyflex.get_positions().reshape(-1, 4)[:, :3],pyflex.get_shape_states()[:3].reshape(-1,3)],axis = 0)
            pyflex.step()

        for j in range(time_step):
            positions[j, :n_particles] = pyflex.get_positions().reshape(-1, 4)[:, :3]
            for k in range(n_shapes):
                 positions[j, n_particles + k] = pyflex.get_shape_states()[:3]
        #    shape_position[j] = pyflex.get_shape_states()[:3].reshape(-1,3)
        #    shape_prevposition = pyflex.get_shape_states()[3:6].reshape(-1,3)
            if j == 0:
                velocities[j] = (positions[j] - p_clip) / dt
           #     shape_velocities[j] = (shape_position[j] - shape_p_clip)/dt
            else:
                velocities[j] = (positions[j] - positions[j - 1]) / dt
          #      shape_velocities[j] = (shape_position[j] - shape_position[j-1])/dt

            pyflex.step()
            data = [positions[j], velocities[j], idx_hairs]
            store_data(data_names, data, os.path.join(rollout_dir, str(j) + '.h5'))
        
        # change dtype for more accurate stat calculation
        # only normalize positions and velocities
        datas = [positions.astype(np.float64), velocities.astype(np.float64)]

        for j in range(len(stats)): 
            # here j = 2, refers to positions and velocities
            stat = init_stat(stats[j].shape[0]) 
            # stat= np.zeros((3,3))
            stat[:, 0] = np.mean(datas[j], axis=(0, 1))[:]
            stat[:, 1] = np.std(datas[j], axis=(0, 1))[:]
            stat[:, 2] = datas[j].shape[0] * datas[j].shape[1] # time_step*n_particles
            stats[j] = combine_stat(stats[j], stat)

    pyflex.clean()

    return stats

In [8]:
data_names = ['positions', 'velocities','hair_idx']
verbose = 0
phase = 'train'
data_dir = os.path.join(args.dataf, phase)
stat_path = os.path.join(args.dataf, 'stat.h5')
n_rollout = 10



info = {
    'env': args.env,
    'root_num': phases_dict['root_num'],
    'thread_idx': 0,
    'data_dir': data_dir,
    'data_names': data_names,
    'n_rollout': n_rollout // args.num_workers,
    'n_instance': args.n_instance,
    'time_step': args.time_step,
    'time_step_clip': args.time_step_clip,
    'dt': args.dt,
    'shape_state_dim': args.shape_state_dim}

info['env_idx'] = 11

In [9]:
class PhysicsFleXDataset(Dataset):

    def __init__(self, args, phase, phases_dict, verbose):
        self.args = args
        self.phase = phase
        self.phases_dict = phases_dict
        self.verbose = verbose
        self.data_dir = os.path.join(self.args.dataf, phase)
        self.stat_path = os.path.join(self.args.dataf, 'stat.h5')

        os.system('mkdir -p ' + self.data_dir)

        #    self.data_names = ['positions', 'velocities', 'shape_quats', 'clusters', 'scene_params']
        self.data_names = ['positions', 'velocities','hair_idx']

        ratio = self.args.train_valid_ratio
        if phase == 'train':
            self.n_rollout = int(self.args.n_rollout * ratio)
        elif phase == 'valid':
            self.n_rollout = self.args.n_rollout - int(self.args.n_rollout * ratio)
        else:
            raise AssertionError("Unknown phase")

    def __len__(self):
        return self.n_rollout * (self.args.time_step - 1)

    def load_data(self, name):
        self.stat = load_data(self.data_names[:2], self.stat_path)
        for i in range(len(self.stat)):
            self.stat[i] = self.stat[i][-self.args.position_dim:, :]
            # print(self.data_names[i], self.stat[i].shape)

    def gen_data(self, name):
        # if the data hasn't been generated, generate the data
        print("Generating data ... n_rollout=%d, time_step=%d" % (self.n_rollout, self.args.time_step))

        infos = []
        for i in range(self.args.num_workers):
            info = {
                'env': self.args.env,
                'root_num': self.phases_dict['root_num'],
                'thread_idx': i,
                'data_dir': self.data_dir,
                'data_names': self.data_names,
                'n_rollout': self.n_rollout // self.args.num_workers,
                'n_instance': self.args.n_instance,
                'time_step': self.args.time_step,
                'time_step_clip': self.args.time_step_clip,
                'dt': self.args.dt,
                'shape_state_dim': self.args.shape_state_dim}

            info['env_idx'] = 11
            infos.append(info)

        cores = self.args.num_workers
        pool = mp.Pool(processes=cores)
        data = pool.map(gen_PyFleX, infos)

        print("Training data generated, warpping up stats ...")

        if self.phase == 'train' and self.args.gen_stat:
            # positions [x, y, z], velocities[xdot, ydot, zdot]
            self.stat = [init_stat(3), init_stat(3)]
            for i in range(len(data)):
                for j in range(len(self.stat)):
                    self.stat[j] = combine_stat(self.stat[j], data[i][j])
            store_data(self.data_names[:2], self.stat, self.stat_path)
        else:
            print("Loading stat from %s ..." % self.stat_path)
            self.stat = load_data(self.data_names[:2], self.stat_path)

    def __getitem__(self, idx):
        idx_rollout = idx // (self.args.time_step - 1)
        idx_timestep = idx % (self.args.time_step - 1)

        # ignore the first frame for env RiceGrip
        if self.args.env == 'RiceGrip' and idx_timestep == 0:
            idx_timestep = np.random.randint(1, self.args.time_step - 1)

        data_path = os.path.join(self.data_dir, str(idx_rollout), str(idx_timestep) + '.h5')
        data_nxt_path = os.path.join(self.data_dir, str(idx_rollout), str(idx_timestep + 1) + '.h5')

        data = load_data(self.data_names, data_path)

        '''
        vel_his = []
        for i in range(self.args.n_his):
            path = os.path.join(self.data_dir, str(idx_rollout), str(max(1, idx_timestep - i - 1)) + '.h5')
            data_his = load_data(self.data_names, path)
            vel_his.append(data_his[1])

        data[1] = np.concatenate([data[1]] + vel_his, 1)
        '''
        
        attr, state, relations, n_particles, n_shapes = \
                prepare_input(data, self.stat, self.args, self.phases_dict, self.verbose)

        ### label
        data_nxt = normalize(load_data(self.data_names, data_nxt_path), self.stat)

        label = torch.FloatTensor(data_nxt[1][:n_particles])

        return attr, state, relations, n_particles, n_shapes, label

In [10]:
def prepare_input(data, stat, args, phases_dict, verbose=0, var=False):
    '''
    for a single hair
    '''
    positions, velocities, hairs_idx = data
    n_shapes = 1
    hairs_idx_begin = [idx[0] for idx in hairs_idx]
    n_particles = positions.shape[0] - n_shapes
    R = 0.1
    
    ### object attributes
    #   dim 10: [rigid, fluid, root_0, root_1, gripper_0, gripper_1, mass_inv,
    #            clusterStiffness, clusterPlasticThreshold, cluasterPlasticCreep]
    #   here we only consider the hairs but not the gripper, attr_dim = 1, attr = 0 for hair, attr = 1 for shapes
    attr = np.zeros((n_particles+n_shapes, args.attr_dim))
    
    ### construct relations
    Rr_idxs = []        # relation receiver idx list
    Rs_idxs = []        # relation sender idx list
    Ras = []            # relation attributes list
    values = []         # relation value list (should be 1)
    node_r_idxs = []    # list of corresponding receiver node idx
    node_s_idxs = []    # list of corresponding sender node idx
    psteps = []         # propagation steps
    
    ##### add env specific graph components
    ### specify for shapes
    rels = []
    vals = []
    
    for i in range(n_shapes):
        attr[n_particles+i, 0] = 1
        dis = np.linalg.norm(positions[:n_particles,:2]-positions[n_particles+i,:2],axis = 1)
        nodes_rel = np.nonzero(dis <= R)[0]
        # for relation between hair nodes and a gripper, we note it as 1
        gripper = np.ones(nodes_rel.shape[0], dtype=np.int) * (n_particles+i)
        rels += [np.stack([nodes_rel, gripper, np.ones(nodes_rel.shape[0])], axis=1)]
        vals += [np.ones(nodes_rel.shape[0], dtype=np.int)]
        
    
    ##### add relations between leaf particles
    ## here we only consider the relations in a hair: the relation between a node and the nodes nearby
    ## simple case for one hair, TEMPORARY 2 rels for one link
    nodes_p = np.arange(n_particles-1)
    val = np.linalg.norm(positions[1:n_particles]-positions[:n_particles-1],axis = 1)
    R1 = np.stack([nodes_p,nodes_p+1, np.zeros(n_particles-1)],axis = 1)
    R2 = np.stack([nodes_p+1,nodes_p, np.zeros(n_particles-1)],axis = 1)
    rels += [np.concatenate([R1,R2],axis = 0)]
    vals += [val,val]
    
    rels = np.concatenate(rels, 0)
    vals = np.concatenate(vals, 0)
    
  #  print (vals.shape)
 #   print (rels.shape)
    
    
    if rels.shape[0] > 0:
        if verbose:
            print("Relations neighbor", rels.shape)
        Rr_idxs.append(torch.LongTensor([rels[:, 0], np.arange(rels.shape[0])]))
        Rs_idxs.append(torch.LongTensor([rels[:, 1], np.arange(rels.shape[0])]))
        # Ra: relation attributes
    #    Ra = np.zeros((rels.shape[0], args.relation_dim))  
        Ra = rels[:,2].reshape([-1,1])
        Ras.append(torch.FloatTensor(Ra))
        # values could be changed
     #   values.append(torch.FloatTensor([1] * rels.shape[0]))
      #  values.append(rels[:,2])
        #### for hairs: values equals to the length of this segment
        values.append(torch.FloatTensor(vals))
        node_r_idxs.append(np.arange(n_particles))
        node_s_idxs.append(np.arange(n_particles + n_shapes))
        psteps.append(args.pstep)
        
        
    if verbose:
        print("Attr shape (after hierarchy building):", attr.shape)
        print("Object attr:", np.sum(attr, axis=0))
        print("Particle attr:", np.sum(attr[:n_particles], axis=0))
        print("Shape attr:", np.sum(attr[n_particles:n_particles+n_shapes], axis=0))
        print("Roots attr:", np.sum(attr[n_particles+n_shapes:], axis=0))
        
        
    ### normalize data
    data = [positions, velocities]
    data = normalize(data, stat, var)
    positions, velocities = data[0], data[1]

    if verbose:
        print("Particle positions stats")
        print(positions.shape)
        print(np.min(positions[:n_particles], 0))
        print(np.max(positions[:n_particles], 0))
        print(np.mean(positions[:n_particles], 0))
        print(np.std(positions[:n_particles], 0))

        show_vel_dim = 6 if args.env == 'RiceGrip' else 3
        print("Velocities stats")
        print(velocities.shape)
        print(np.mean(velocities[:n_particles, :show_vel_dim], 0))
        print(np.std(velocities[:n_particles, :show_vel_dim], 0))
        
    state = torch.FloatTensor(np.concatenate([positions, velocities], axis=1))
    attr = torch.FloatTensor(attr)
    relations = [Rr_idxs, Rs_idxs, values, Ras, node_r_idxs, node_s_idxs, psteps]

    return attr, state, relations, n_particles, n_shapes#, instance_idx


    
    

In [11]:
datasets = {phase: PhysicsFleXDataset(
    args, phase, phases_dict, args.verbose_data) for phase in ['train', 'valid']}


In [12]:
datasets['train'].load_data(args.env)

In [13]:
data = datasets['train'].__getitem__(1)

Relations neighbor (60, 3)
Attr shape (after hierarchy building): (32, 1)
Object attr: [1.]
Particle attr: [0.]
Shape attr: [1.]
Roots attr: [0.]
Particle positions stats
(32, 3)
[ 0.39366985 -1.84051862 -0.21294536]
[ 0.39366985  1.72843141 -0.21294536]
[ 0.39366985 -0.07940947 -0.21294536]
[1.66533454e-16 1.06529481e+00 1.66533454e-16]
Velocities stats
(32, 3)
[-0.08079876  3.02442785  0.01608018]
[4.16333634e-17 1.54151227e+00 6.93889390e-18]


In [14]:
attr, state, rels, n_particles, n_shapes, label = data

In [15]:
label.shape

torch.Size([31, 3])

In [16]:
rels

[[tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
           18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29,  1,  2,  3,  4,  5,  6,
            7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24,
           25, 26, 27, 28, 29, 30],
          [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
           18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
           36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
           54, 55, 56, 57, 58, 59]])],
 [tensor([[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
           19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30,  0,  1,  2,  3,  4,  5,
            6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23,
           24, 25, 26, 27, 28, 29],
          [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
           18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 

In [17]:
dataloaders = {x: torch.utils.data.DataLoader(
    datasets[x], batch_size=args.batch_size,
    shuffle=True if x == 'train' else False,
    num_workers=args.num_workers,
    collate_fn=collate_fn)
    for x in ['train', 'valid']}

In [18]:
phase = 'train'
for i, data in enumerate(dataloaders[phase]):
    break


Relations neighborRelations neighbor Relations neighbor (60, 3) 
(60, 3)(60, 3)Attr shape (after hierarchy building):

Attr shape (after hierarchy building): Attr shape (after hierarchy building): (32, 1)(32, 1)

Object attr: [1.]
Particle attr: [0.]
Shape attr: [1.]
Roots attr: [0.]
Particle positions stats Object attr:(32, 1) 

(32, 3)
[1.][-1.74801819 -1.60031738 -0.29082427]

Object attr: [1.]
Particle attr: [0.]
Shape attr:Particle attr:[0.39366985 1.72843141 2.02063993]
 [-0.92047091  0.06530966  0.76834979][1.] 

[0.]Roots attr:
[0.77081759 0.9801186  0.65969195] Shape attr:[0.]
 Velocities stats

(32, 3)[1.]Particle positions stats

Roots attr:(32, 3) 
[ 0.39366985 -1.66066061 -0.21294536][0.]

[ 0.39366985  1.72843141 -0.21294536]
Particle positions stats[ 0.39366985  0.03547397 -0.21294536]


[-1.16159317  0.43273923  0.00167318](32, 3)

[1.66533454e-16 1.00903497e+00 1.66533454e-16]
[ 0.39366985 -1.68000499 -0.21294536][1.87052081 0.39230342 0.63644768]Velocities stats


(32

In [19]:
attr, state, rels, n_particles, n_shapes, label = data
Ra, node_r_idx, node_s_idx, pstep = rels[3], rels[4], rels[5], rels[6]

In [20]:
Rr, Rs = [], []
for j in range(len(rels[0])):
    Rr_idx, Rs_idx, values = rels[0][j], rels[1][j], rels[2][j]
    Rr.append(torch.sparse.FloatTensor(Rr_idx, values, torch.Size([node_r_idx[j].shape[0], Ra[j].size(0)])))
    Rs.append(torch.sparse.FloatTensor(Rs_idx, values, torch.Size([node_s_idx[j].shape[0], Ra[j].size(0)])))


In [21]:
data2 = [attr, state, Rr, Rs, Ra, label]

In [38]:
instance_idx = [0,31]

In [39]:
args.pstep = 1
pstep = 1
use_gpu = False
model = DPINet(args, datasets['train'].stat, phases_dict, residual=True, use_gpu=use_gpu)
print("Number of parameters: %d" % count_parameters(model))

Number of parameters: 615210


In [40]:
# criterion
criterionMSE = nn.MSELoss()

# optimizer
optimizer = optim.Adam(model.parameters(), lr=args.lr, betas=(args.beta1, 0.999))
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.8, patience=3, verbose=True)

if use_gpu:
    model = model.cuda()
    criterionMSE = criterionMSE.cuda()

st_epoch = args.resume_epoch if args.resume_epoch > 0 else 0
best_valid_loss = np.inf

In [41]:
model.train(phase=='train')

DPINet(
  (particle_encoder_list): ModuleList(
    (0): ParticleEncoder(
      (model): Sequential(
        (0): Linear(in_features=13, out_features=200, bias=True)
        (1): ReLU()
        (2): Linear(in_features=200, out_features=200, bias=True)
        (3): ReLU()
      )
    )
  )
  (relation_encoder_list): ModuleList(
    (0): RelationEncoder(
      (model): Sequential(
        (0): Linear(in_features=27, out_features=300, bias=True)
        (1): ReLU()
        (2): Linear(in_features=300, out_features=300, bias=True)
        (3): ReLU()
        (4): Linear(in_features=300, out_features=300, bias=True)
        (5): ReLU()
      )
    )
  )
  (relation_propagator_list): ModuleList(
    (0): Propagator(
      (linear): Linear(in_features=700, out_features=200, bias=True)
      (relu): ReLU()
    )
  )
  (particle_propagator_list): ModuleList(
    (0): Propagator(
      (linear): Linear(in_features=400, out_features=200, bias=True)
      (relu): ReLU()
    )
  )
  (rigid_particle_

In [43]:
pstep = [1]

In [44]:
predicted = model(
                    attr, state, Rr, Rs, Ra, n_particles,
                    node_r_idx, node_s_idx, pstep,
                    instance_idx, phases_dict, args.verbose_model)

=== Stage 0
Rrp tensor(indices=tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,
                        14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,
                        28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41,
                        42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55,
                        56, 57, 58, 59],
                       [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,
                        14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,
                        28, 29,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12,
                        13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
                        27, 28, 29, 30]]),
       values=tensor([0.1108, 0.1102, 0.1119, 0.1113, 0.1112, 0.1110, 0.1109,
                      0.1107, 0.1106, 0.1104, 0.1103, 0.1101, 0.1099, 0.1098,
                      0.1096, 0.1095, 0.1094, 0.1092, 0.1090, 0.1096, 0.1095,
                      0.10

RuntimeError: There were no tensor arguments to this function (e.g., you passed an empty list of Tensors), but no fallback function is registered for schema aten::_cat.  This usually means that this function requires a non-empty list of Tensors.  Available functions are [CPU, CUDA, QuantizedCPU, BackendSelect, Named, AutogradOther, AutogradCPU, AutogradCUDA, AutogradXLA, AutogradPrivateUse1, AutogradPrivateUse2, AutogradPrivateUse3, Tracer, Autocast, Batched, VmapMode].

CPU: registered at /pytorch/build/aten/src/ATen/CPUType.cpp:2127 [kernel]
CUDA: registered at /pytorch/build/aten/src/ATen/CUDAType.cpp:2983 [kernel]
QuantizedCPU: registered at /pytorch/build/aten/src/ATen/QuantizedCPUType.cpp:297 [kernel]
BackendSelect: fallthrough registered at /pytorch/aten/src/ATen/core/BackendSelectFallbackKernel.cpp:3 [backend fallback]
Named: registered at /pytorch/aten/src/ATen/core/NamedRegistrations.cpp:7 [backend fallback]
AutogradOther: registered at /pytorch/torch/csrc/autograd/generated/

In [133]:
class DPINet(nn.Module):
    def __init__(self, args, stat, phases_dict, residual=False, use_gpu=False):
        super(DPINet, self).__init__()

        self.args = args

        state_dim = args.state_dim
        attr_dim = args.attr_dim
        relation_dim = args.relation_dim
        nf_particle = 20#args.nf_particle
        nf_relation = 30 #args.nf_relation
        nf_effect = 30 #args.nf_effect

        self.nf_effect = nf_effect

        self.stat = stat
        self.use_gpu = use_gpu
        self.residual = residual 

        # (1) particle attr (2) state
        self.particle_encoder = ParticleEncoder(attr_dim + state_dim, nf_particle, nf_effect)

        # (1) sender attr (2) receiver attr (3) state receiver (4) state_diff (5) relation attr
        self.relation_encoder = RelationEncoder( 2 * attr_dim + 2 * state_dim + relation_dim, nf_relation, nf_relation)

        # (1) relation encode (2) sender effect (3) receiver effect
        self.relation_propagator = Propagator(nf_relation + 2 * nf_effect, nf_effect)

        # (1) particle encode (2) particle effect
        self.particle_propagator = Propagator(2 * nf_effect, nf_effect)

        # (1) set particle effect
        self.particle_predictor = ParticlePredictor(nf_effect, nf_effect, args.position_dim)
        
        
    def forward(self, attr, state, Rr, Rs, Ra, n_particles, node_r_idx, node_s_idx, pstep,
                instance_idx, phases_dict, verbose=0):
        
        # calculate particle encoding
        if self.use_gpu:
            particle_effect = Variable(torch.zeros((attr.size(0), self.nf_effect)).cuda())
        else:
            particle_effect = Variable(torch.zeros((attr.size(0), self.nf_effect)))
            
        s = 0
        Rrp = Rr[s].t()
        Rsp = Rs[s].t()

        # receiver_attr, sender_attr
        attr_r = attr[node_r_idx[s]]
        attr_s = attr[node_s_idx[s]]
        attr_r_rel = Rrp.mm(attr_r)
        attr_s_rel = Rsp.mm(attr_s)

        # receiver_state, sender_state
        state_r = state[node_r_idx[s]]
        state_s = state[node_s_idx[s]]
        state_r_rel = Rrp.mm(state_r)
        state_s_rel = Rsp.mm(state_s)
        state_diff = state_r_rel - state_s_rel
        
        particle_encode = self.particle_encoder(torch.cat([attr_r, state_r], 1))
        relation_encode = self.relation_encoder(
                torch.cat([attr_r_rel, attr_s_rel, state_r_rel, state_s_rel, Ra[s]], 1))
        
        for i in range(pstep):
            effect_p_r = particle_effect[node_r_idx[s]]
            effect_p_s = particle_effect[node_s_idx[s]]

            receiver_effect = Rrp.mm(effect_p_r)
            sender_effect = Rsp.mm(effect_p_s)
            

            # calculate relation effect
            effect_rel = self.relation_propagator(
                torch.cat([relation_encode, receiver_effect, sender_effect], 1))

            # calculate particle effect by aggregating relation effect
            effect_p_r_agg = Rr[s].mm(effect_rel)

            # calculate particle effect
            effect_p = self.particle_propagator(
                torch.cat([particle_encode, effect_p_r_agg], 1),
                res=effect_p_r)
            particle_effect[node_r_idx[s]] = effect_p
            
        pred = self.particle_predictor(particle_effect[:31])
        return pred
        
        
        
        
        
        
        
        

In [134]:
args.pstep = 3
pstep = 3
use_gpu = False
model = DPINet(args, datasets['train'].stat, phases_dict, residual=True, use_gpu=use_gpu)
print("Number of parameters: %d" % count_parameters(model))
# criterion
criterionMSE = nn.MSELoss()

# optimizer
optimizer = optim.Adam(model.parameters(), lr=args.lr, betas=(args.beta1, 0.999))
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.8, patience=3, verbose=True)

if use_gpu:
    model = model.cuda()
    criterionMSE = criterionMSE.cuda()

st_epoch = args.resume_epoch if args.resume_epoch > 0 else 0
best_valid_loss = np.inf

model.train(phase=='train')

Number of parameters: 9643


DPINet(
  (particle_encoder): ParticleEncoder(
    (model): Sequential(
      (0): Linear(in_features=7, out_features=20, bias=True)
      (1): ReLU()
      (2): Linear(in_features=20, out_features=30, bias=True)
      (3): ReLU()
    )
  )
  (relation_encoder): RelationEncoder(
    (model): Sequential(
      (0): Linear(in_features=15, out_features=30, bias=True)
      (1): ReLU()
      (2): Linear(in_features=30, out_features=30, bias=True)
      (3): ReLU()
      (4): Linear(in_features=30, out_features=30, bias=True)
      (5): ReLU()
    )
  )
  (relation_propagator): Propagator(
    (linear): Linear(in_features=90, out_features=30, bias=True)
    (relu): ReLU()
  )
  (particle_propagator): Propagator(
    (linear): Linear(in_features=60, out_features=30, bias=True)
    (relu): ReLU()
  )
  (particle_predictor): ParticlePredictor(
    (linear_0): Linear(in_features=30, out_features=30, bias=True)
    (linear_1): Linear(in_features=30, out_features=30, bias=True)
    (linear_2): Li

In [135]:
predicted = model(attr, state, Rr, Rs, Ra, n_particles,
                    node_r_idx, node_s_idx, pstep,
                    instance_idx, phases_dict, args.verbose_model)

In [136]:
predicted

tensor([[ 0.0072,  0.0273, -0.0420],
        [ 0.0068,  0.0268, -0.0411],
        [ 0.0065,  0.0271, -0.0418],
        [ 0.0064,  0.0276, -0.0431],
        [ 0.0064,  0.0278, -0.0439],
        [ 0.0064,  0.0278, -0.0441],
        [ 0.0066,  0.0279, -0.0442],
        [ 0.0068,  0.0279, -0.0442],
        [ 0.0068,  0.0278, -0.0443],
        [ 0.0061,  0.0274, -0.0453],
        [ 0.0059,  0.0273, -0.0457],
        [ 0.0067,  0.0280, -0.0455],
        [ 0.0078,  0.0304, -0.0454],
        [ 0.0079,  0.0309, -0.0446],
        [ 0.0075,  0.0293, -0.0451],
        [ 0.0064,  0.0278, -0.0451],
        [ 0.0060,  0.0273, -0.0454],
        [ 0.0066,  0.0287, -0.0448],
        [ 0.0076,  0.0302, -0.0436],
        [ 0.0077,  0.0293, -0.0438],
        [ 0.0081,  0.0286, -0.0438],
        [ 0.0080,  0.0284, -0.0435],
        [ 0.0081,  0.0284, -0.0427],
        [ 0.0090,  0.0290, -0.0418],
        [ 0.0113,  0.0303, -0.0406],
        [ 0.0124,  0.0308, -0.0402],
        [ 0.0123,  0.0302, -0.0391],
 

In [ ]:
phase = 'train'

for epoch in range(100):
    model.train(phase=='train')

    losses = 0.
    for i, data in enumerate(dataloaders[phase]):
#         print ('i:',i)

        attr, state, rels, n_particles, n_shapes, label = data
        Ra, node_r_idx, node_s_idx, pstep = rels[3], rels[4], rels[5], rels[6]

        Rr, Rs = [], []
        for j in range(len(rels[0])):
            Rr_idx, Rs_idx, values = rels[0][j], rels[1][j], rels[2][j]
            Rr.append(torch.sparse.FloatTensor(
                Rr_idx, values, torch.Size([node_r_idx[j].shape[0], Ra[j].size(0)])))
            Rs.append(torch.sparse.FloatTensor(
                Rs_idx, values, torch.Size([node_s_idx[j].shape[0], Ra[j].size(0)])))

        data = [attr, state, Rr, Rs, Ra, label]
        instance_idx = [0, 31]

            # st_time = time.time()
        pstep = 3
        predicted = model(
            attr, state, Rr, Rs, Ra, n_particles,
            node_r_idx, node_s_idx, pstep,
            instance_idx, phases_dict, 0)
            # print('Time forward', time.time() - st_time)

       #     print(predicted.shape)
       #     print(label.shape)

        loss = criterionMSE(predicted, label)
   #     print ("loss",loss)
        losses += np.sqrt(loss.item())

        if phase == 'train':
            if i % 1 == 0:
                # update parameters every args.forward_times
          #      print ('update!')
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
          #      print ('done')


        if i % 10 == 0:
            n_relations = 0
            for j in range(len(Ra)):
                n_relations += Ra[j].size(0)
            print('%s [%d/%d][%d/%d] n_relations: %d, Loss: %.6f, Agg: %.6f' %
                  (phase, epoch, args.n_epoch, i, len(dataloaders[phase]),
                   n_relations, np.sqrt(loss.item()), losses / (i + 1)))

     #   if phase == 'train' and i > 0 and i % args.ckp_per_iter == 0:
     #       torch.save(model.state_dict(), '%s/net_epoch_%d_iter_%d.pth' % (args.outf, epoch, i))

    losses /= len(dataloaders[phase])
    print('%s [%d/%d] Loss: %.4f, Best valid: %.4f' %
          (phase, epoch, args.n_epoch, losses, best_valid_loss))


In [ ]:
    def forward(self, attr, state, Rr, Rs, Ra, n_particles, node_r_idx, node_s_idx, pstep,
                instance_idx, phases_dict, verbose=0):

        # calculate particle encoding
        if self.use_gpu:
            particle_effect = Variable(torch.zeros((attr.size(0), self.nf_effect)).cuda())
        else:
            particle_effect = Variable(torch.zeros((attr.size(0), self.nf_effect)))

        # add offset to center-of-mass for rigids to attr
        if self.use_gpu:
            offset = Variable(torch.zeros((attr.size(0), state.size(1))).cuda())
        else:
            offset = Variable(torch.zeros((attr.size(0), state.size(1))))

        for i in range(len(instance_idx) - 1):
            st, ed = instance_idx[i], instance_idx[i + 1]
            if phases_dict['material'][i] == 'rigid':
                c = torch.mean(state[st:ed], dim=0)
                offset[st:ed] = state[st:ed] - c
        attr = torch.cat([attr, offset], 1)

        n_stage = len(Rr)
        for s in range(n_stage):
            if verbose:
                print("=== Stage", s)
            Rrp = Rr[s].t()
            Rsp = Rs[s].t()

            if verbose:
                print ('Rrp',Rrp)

            # receiver_attr, sender_attr
            attr_r = attr[node_r_idx[s]]
            attr_s = attr[node_s_idx[s]]
            attr_r_rel = Rrp.mm(attr_r)
            attr_s_rel = Rsp.mm(attr_s)

            # receiver_state, sender_state
            state_r = state[node_r_idx[s]]
            state_s = state[node_s_idx[s]]
            state_r_rel = Rrp.mm(state_r)
            state_s_rel = Rsp.mm(state_s)
            state_diff = state_r_rel - state_s_rel

            # particle encode
            if verbose:
                print('attr_r', attr_r.shape, 'state_r', state_r.shape)
            particle_encode = self.particle_encoder_list[s](torch.cat([attr_r, state_r], 1))

            # calculate relation encoding
            relation_encode = self.relation_encoder_list[s](
                torch.cat([attr_r_rel, attr_s_rel, state_r_rel, state_s_rel, Ra[s]], 1))
            if verbose:
                print ('attr_r_rel', attr_r_rel.shape, 'attr_s_rel', attr_s_rel.shape, 'state_r_rel', state_r_rel.shape, 'state_s_rel', state_s_rel.shape, 'Ra[s]',  Ra[s].shape)
                print("relation encode:", relation_encode.size())

            for i in range(pstep[s]):
                if verbose:
                    print("pstep", i)
                    print("Receiver index range", np.min(node_r_idx[s]), np.max(node_r_idx[s]))
                    print("Sender index range", np.min(node_s_idx[s]), np.max(node_s_idx[s]))

                effect_p_r = particle_effect[node_r_idx[s]]
                effect_p_s = particle_effect[node_s_idx[s]]

                receiver_effect = Rrp.mm(effect_p_r)
                sender_effect = Rsp.mm(effect_p_s)

                # calculate relation effect
                effect_rel = self.relation_propagator_list[s](
                    torch.cat([relation_encode, receiver_effect, sender_effect], 1))
                if verbose:
                    print("relation effect:", effect_rel.size())

                # calculate particle effect by aggregating relation effect
                effect_p_r_agg = Rr[s].mm(effect_rel)

                # calculate particle effect
                effect_p = self.particle_propagator_list[s](
                    torch.cat([particle_encode, effect_p_r_agg], 1),
                    res=effect_p_r)
                if verbose:
                    print("particle effect:", effect_p.size())

                particle_effect[node_r_idx[s]] = effect_p

        pred = []
        for i in range(len(instance_idx) - 1):
            st, ed = instance_idx[i], instance_idx[i + 1]

            if phases_dict['material'][i] == 'rigid':
                t = self.rigid_particle_predictor(torch.mean(particle_effect[st:ed], 0)).view(-1)

                R = self.rotation_matrix_from_quaternion(t[:4])
                b = t[4:] * self.std_p

                p_0 = state[st:ed, :3] * self.std_p + self.mean_p
                c = torch.mean(p_0, dim=0)
                p_1 = torch.mm(p_0 - c, R) + b + c
                v = (p_1 - p_0) / self.dt
                pred.append((v - self.mean_v) / self.std_v)

            elif phases_dict['material'][i] == 'fluid':
                pred.append(self.fluid_particle_predictor(particle_effect[st:ed]))

        pred = torch.cat(pred, 0)

        if verbose:
            print("pred:", pred.size())

        return pred


In [78]:
from models import *

attr_dim = args.attr_dim
state_dim = args.state_dim
relation_dim = args.relation_dim
nf_particle = args.nf_particle
nf_relation = 30 #args.nf_relation
nf_effect = 20 #args.nf_effect



# (1) particle attr (2) state
particle_encoder = ParticleEncoder(attr_dim + state_dim, nf_particle, nf_effect)

# (1) sender attr (2) receiver attr (3) state receiver (4) state_diff (5) relation attr
relation_encoder = RelationEncoder( 2 * attr_dim + 2 * state_dim + relation_dim, nf_relation, nf_relation)

# (1) relation encode (2) sender effect (3) receiver effect
relation_propagator = Propagator(nf_relation + 2 * nf_effect, nf_effect)

# (1) particle encode (2) particle effect
particle_propagator = Propagator(2 * nf_effect, nf_effect)

# (1) set particle effect
rigid_particle_predictor = ParticlePredictor(nf_effect, nf_effect, 7)  # predict rigid motion
fluid_particle_predictor = ParticlePredictor(nf_effect, nf_effect, args.position_dim)

In [79]:
2 * attr_dim + 2* state_dim + relation_dim

15

In [59]:
Ra[0].shape

torch.Size([60, 1])

In [39]:
Rr, Rs = [], []
Rrv, Rsv = [], []
j = 0
Rr_idx, Rs_idx, values = rels[0][j], rels[1][j], rels[2][j]
V = torch.ones(values.shape)
Rr.append(torch.sparse.FloatTensor(Rr_idx, V, torch.Size([node_r_idx[j].shape[0], Ra[j].size(0)])))
Rs.append(torch.sparse.FloatTensor(Rs_idx, V, torch.Size([node_s_idx[j].shape[0], Ra[j].size(0)])))
Rrv.append(torch.sparse.FloatTensor(Rr_idx, values, torch.Size([node_r_idx[j].shape[0], Ra[j].size(0)])))
Rsv.append(torch.sparse.FloatTensor(Rs_idx, values, torch.Size([node_s_idx[j].shape[0], Ra[j].size(0)])))


In [65]:
Rr[0].shape

torch.Size([31, 60])

In [71]:
values.shape

torch.Size([60])

In [74]:
Rr[0].mm(values.reshape([60,1])).shape

torch.Size([31, 1])

In [50]:
Rs_idx

tensor([[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
         19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30,  0,  1,  2,  3,  4,  5,
          6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23,
         24, 25, 26, 27, 28, 29],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
         36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
         54, 55, 56, 57, 58, 59]])

In [35]:
values.shape

torch.Size([60])

In [34]:
Rr_idx

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29,  1,  2,  3,  4,  5,  6,
          7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24,
         25, 26, 27, 28, 29, 30],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
         36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
         54, 55, 56, 57, 58, 59]])

In [40]:
s = 0
Rrp = Rr[s].t()
Rsp = Rs[s].t()

# receiver_attr, sender_attr
attr_r = attr[node_r_idx[s]]
attr_s = attr[node_s_idx[s]]
attr_r_rel = Rrp.mm(attr_r)
attr_s_rel = Rsp.mm(attr_s)

# receiver_state, sender_state
state_r = state[node_r_idx[s]]
state_s = state[node_s_idx[s]]
state_r_rel = Rrp.mm(state_r)
state_s_rel = Rsp.mm(state_s)
state_diff = state_r_rel - state_s_rel

In [54]:
Rrp

tensor(indices=tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,
                        14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,
                        28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41,
                        42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55,
                        56, 57, 58, 59],
                       [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,
                        14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,
                        28, 29,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12,
                        13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
                        27, 28, 29, 30]]),
       values=tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                      1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                      1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                      1., 1., 1., 1., 1., 

In [53]:
state_s_rel.shape

torch.Size([60, 6])

In [45]:
attr_s

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.]])

In [46]:
attr_s_rel

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]])

In [42]:
attr_r_rel.shape

torch.Size([60, 1])

In [31]:
node_r_idx[s]

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30])

In [28]:
Rs[s].shape

torch.Size([32, 60])

In [30]:
attr_r_rel.shape

torch.Size([60, 1])

In [27]:
Rr[s]


tensor(indices=tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,
                        14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,
                        28, 29,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12,
                        13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
                        27, 28, 29, 30],
                       [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,
                        14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,
                        28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41,
                        42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55,
                        56, 57, 58, 59]]),
       values=tensor([0.1079, 0.1076, 0.1078, 0.1076, 0.1073, 0.1073, 0.1072,
                      0.1071, 0.1070, 0.1070, 0.1069, 0.1069, 0.1069, 0.1068,
                      0.1068, 0.1068, 0.1068, 0.1068, 0.1069, 0.1069, 0.1069,
                      0.1070, 0.1070, 0.10

In [24]:
print(attr_r.shape)
attr_r

torch.Size([31, 1])


tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]])

In [25]:
print(attr_s.shape)
attr_s

torch.Size([32, 1])


tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.]])

In [81]:
print(attr_r.shape)
print(state_r.shape)

torch.Size([31, 1])
torch.Size([31, 6])


In [82]:
particle_encode = particle_encoder(torch.cat([attr_r, state_r], 1))

In [83]:
particle_encode.shape

torch.Size([31, 20])

In [84]:
relation_encode = relation_encoder(
                torch.cat([attr_r_rel, attr_s_rel, state_r_rel, state_s_rel, Ra[s]], 1))

In [85]:
relation_encode.shape

torch.Size([60, 30])

In [88]:
s = 0
pstep = 3
particle_effect = Variable(torch.zeros((attr.size(0), nf_effect)))

effect_p_r = particle_effect[node_r_idx[s]]
effect_p_s = particle_effect[node_s_idx[s]]

receiver_effect = Rrp.mm(effect_p_r)
sender_effect = Rsp.mm(effect_p_s)

In [90]:
relation_propagator(
        torch.cat([relation_encode, receiver_effect, sender_effect], 1))

tensor([[0.0717, 0.0309, 0.0000,  ..., 0.0000, 0.0547, 0.0539],
        [0.0718, 0.0312, 0.0000,  ..., 0.0000, 0.0537, 0.0550],
        [0.0719, 0.0307, 0.0000,  ..., 0.0000, 0.0534, 0.0544],
        ...,
        [0.0711, 0.0317, 0.0000,  ..., 0.0000, 0.0551, 0.0545],
        [0.0713, 0.0312, 0.0000,  ..., 0.0000, 0.0542, 0.0543],
        [0.0722, 0.0307, 0.0000,  ..., 0.0000, 0.0530, 0.0540]],
       grad_fn=<ReluBackward0>)

In [92]:
s = 0
pstep = 3
particle_effect = Variable(torch.zeros((attr.size(0), nf_effect)))

for i in range(pstep):
    effect_p_r = particle_effect[node_r_idx[s]]
    effect_p_s = particle_effect[node_s_idx[s]]

    receiver_effect = Rrp.mm(effect_p_r)
    sender_effect = Rsp.mm(effect_p_s)

    # calculate relation effect
    effect_rel = relation_propagator(
        torch.cat([relation_encode, receiver_effect, sender_effect], 1))

    # calculate particle effect by aggregating relation effect
    effect_p_r_agg = Rr[s].mm(effect_rel)

    # calculate particle effect
    effect_p = particle_propagator(
        torch.cat([particle_encode, effect_p_r_agg], 1),
        res=effect_p_r)
    particle_effect[node_r_idx[s]] = effect_p

In [98]:
fluid_particle_predictor(particle_effect[:31]).shape

torch.Size([31, 3])

In [53]:
torch.cat([attr_r_rel, attr_s_rel, state_r_rel, state_s_rel, Ra[0].reshape([-1,1])], 1)

tensor([[ 0.0000,  0.0000,  0.0437,  0.1919, -0.0236, -0.0090, -0.0035,  0.0018,
          0.0346,  0.1793, -0.0716, -0.0090, -0.0035,  0.0018,  0.0000],
        [ 0.0000,  0.0000,  0.0344,  0.1783, -0.0712, -0.0089, -0.0034,  0.0018,
          0.0252,  0.1659, -0.0523, -0.0087, -0.0028, -0.1887,  0.0000],
        [ 0.0000,  0.0000,  0.0256,  0.1685, -0.0532, -0.0089, -0.0028, -0.1917,
          0.0158,  0.1557, -0.0518, -0.0087, -0.0021, -0.3785,  0.0000],
        [ 0.0000,  0.0000,  0.0157,  0.1549, -0.0515, -0.0087, -0.0021, -0.3764,
          0.0055,  0.1423, -0.0104, -0.0028, -0.0033, -0.0802,  0.0000],
        [ 0.0000,  0.0000,  0.0055,  0.1422, -0.0104, -0.0028, -0.0033, -0.0801,
         -0.0053,  0.1297,  0.0124,  0.0016, -0.0040, -0.0664,  0.0000],
        [ 0.0000,  0.0000, -0.0053,  0.1295,  0.0124,  0.0016, -0.0040, -0.0662,
         -0.0167,  0.1172,  0.0378,  0.0069, -0.0051, -0.0679,  0.0000],
        [ 0.0000,  0.0000, -0.0167,  0.1170,  0.0378,  0.0069, -0.0051, -0.0

In [48]:
attr_s_rel.shape

torch.Size([60, 1])

In [52]:
Ra[0].reshape([-1,1]).shape

torch.Size([60, 1])

In [ ]:
d = gen_PyFleX(info)

In [ ]:
stats, datas = d

In [ ]:
stats

In [ ]:
positions, velocities, hair_idx, shape_position = datas

In [ ]:
def prepare_input(data, stat, args, phases_dict, verbose=0, var=False):
    '''
    for a single hair
    '''
    positions, velocities, hairs_idx, shape_position = data
    n_shapes = 1
    hairs_idx_begin = [idx[0] for idx in hairs_idx]
    n_particles = positions.shape[0]
    R = 0.1
    R = 2
    
    ### object attributes
    #   dim 10: [rigid, fluid, root_0, root_1, gripper_0, gripper_1, mass_inv,
    #            clusterStiffness, clusterPlasticThreshold, cluasterPlasticCreep]
    #   here we only consider the hairs but not the gripper, attr_dim = 1, attr = 0 for hair, attr = 1 for shapes
    attr = np.zeros((n_particles+n_shapes, args.attr_dim))
    
    ### construct relations
    Rr_idxs = []        # relation receiver idx list
    Rs_idxs = []        # relation sender idx list
    Ras = []            # relation attributes list
    values = []         # relation value list (should be 1)
    node_r_idxs = []    # list of corresponding receiver node idx
    node_s_idxs = []    # list of corresponding sender node idx
    psteps = []         # propagation steps
    
    ##### add env specific graph components
    ### specify for shapes
    rels = []
    vals = []
    
    for i in range(n_shapes):
        attr[n_particles+i, 0] = 1
        dis = np.linalg.norm(positions[:,:2]- shape_position[0,:2],axis = 1)
        nodes_rel = np.nonzero(dis <= R)[0]
        # for relation between hair nodes and a gripper, we note it as 1
        gripper = np.ones(nodes_rel.shape[0], dtype=np.int) * (n_particles+i)
        rels += [np.stack([nodes_rel, gripper, np.ones(nodes_rel.shape[0])], axis=1)]
        vals += [np.ones(nodes_rel.shape[0], dtype=np.int)]
        
    
    ##### add relations between leaf particles
    ## here we only consider the relations in a hair: the relation between a node and the nodes nearby
    ## simple case for one hair, TEMPORARY 2 rels for one link
    nodes_p = np.arange(n_particles-1)
    val = np.linalg.norm(positions[1:]-positions[:-1],axis = 1)
    R1 = np.stack([nodes_p,nodes_p+1, np.zeros(n_particles-1)],axis = 1)
    R2 = np.stack([nodes_p+1,nodes_p, np.zeros(n_particles-1)],axis = 1)
    rels += [np.concatenate([R1,R2],axis = 0)]
    vals += [val,val]
    
    rels = np.concatenate(rels, 0)
    vals = np.concatenate(vals, 0)
    
  #  print (vals.shape)
 #   print (rels.shape)
    
    
    if rels.shape[0] > 0:
        if verbose:
            print("Relations neighbor", rels.shape)
        Rr_idxs.append(torch.LongTensor([rels[:, 0], np.arange(rels.shape[0])]))
        Rs_idxs.append(torch.LongTensor([rels[:, 1], np.arange(rels.shape[0])]))
        # Ra: relation attributes
    #    Ra = np.zeros((rels.shape[0], args.relation_dim))  
        Ra = rels[:,2]
        Ras.append(torch.FloatTensor(Ra))
        # values could be changed
     #   values.append(torch.FloatTensor([1] * rels.shape[0]))
      #  values.append(rels[:,2])
        #### for hairs: values equals to the length of this segment
        values.append(vals)
        node_r_idxs.append(np.arange(n_particles))
        node_s_idxs.append(np.arange(n_particles + n_shapes))
        psteps.append(args.pstep)
        
        
    if verbose:
        print("Attr shape (after hierarchy building):", attr.shape)
        print("Object attr:", np.sum(attr, axis=0))
        print("Particle attr:", np.sum(attr[:n_particles], axis=0))
        print("Shape attr:", np.sum(attr[n_particles:n_particles+n_shapes], axis=0))
        print("Roots attr:", np.sum(attr[n_particles+n_shapes:], axis=0))
        
        
    ### normalize data
    data = [positions, velocities]
    data = normalize(data, stat, var)
    positions, velocities = data[0], data[1]

    if verbose:
        print("Particle positions stats")
        print(positions.shape)
        print(np.min(positions[:n_particles], 0))
        print(np.max(positions[:n_particles], 0))
        print(np.mean(positions[:n_particles], 0))
        print(np.std(positions[:n_particles], 0))

        show_vel_dim = 6 if args.env == 'RiceGrip' else 3
        print("Velocities stats")
        print(velocities.shape)
        print(np.mean(velocities[:n_particles, :show_vel_dim], 0))
        print(np.std(velocities[:n_particles, :show_vel_dim], 0))
        
    state = torch.FloatTensor(np.concatenate([positions, velocities], axis=1))
    attr = torch.FloatTensor(attr)
    relations = [Rr_idxs, Rs_idxs, values, Ras, node_r_idxs, node_s_idxs, psteps]

    return attr, state, relations, n_particles, n_shapes#, instance_idx


    
    
    
    

In [ ]:
attr, state, relations, n_particles, n_shapes = prepare_input(data, stat, args, phases_dict)

In [ ]:
Rr_idxs, Rs_idxs, values, Ras, node_r_idxs, node_s_idxs, psteps = relations

In [ ]:
Rs_idxs

In [ ]:
values

In [ ]:
Ras

In [ ]:
def prepare_input(data, stat, args, phases_dict, verbose=0, var=False):

    # Arrangement:
    # particles, shapes, roots

    if args.env == 'RiceGrip':
        positions, velocities, shape_quats, clusters, scene_params = data
        n_shapes = shape_quats.size(0) if var else shape_quats.shape[0]
    elif args.env == 'FluidShake':
        positions, velocities, shape_quats, scene_params = data
        n_shapes = shape_quats.size(0) if var else shape_quats.shape[0]
        clusters = None
    elif args.env == 'BoxBath':
        positions, velocities, clusters = data
        n_shapes = 0
    elif args.env == 'FluidFall':
        positions, velocities = data
        n_shapes = 0
        clusters = None
    elif args.env == 'Hairs':
        positions, velocities, hairs_idx = data
        n_shapes = 1
        hairs_idx_begin = [idx[0] for idx in hairs_idx]
        clusters = None

    count_nodes = positions.size(0) if var else positions.shape[0]
    n_particles = count_nodes - n_shapes

    if verbose:
        print("positions", positions.shape)
        print("velocities", velocities.shape)

        print("n_particles", n_particles)
        print("n_shapes", n_shapes)
        if args.env == 'RiceGrip' or args.env == 'FluidShake':
            print("shape_quats", shape_quats.shape)

    ### instance idx
    #   instance_idx (n_instance + 1): start idx of instance
    if args.env == 'RiceGrip' or args.env == 'FluidShake':
        instance_idx = [0, n_particles]
    elif args.env == 'Hairs':
        instance_idx = [0, count_nodes]
    else:
        instance_idx = phases_dict["instance_idx"]
    if verbose:
        print("Instance_idx:", instance_idx)


    ### object attributes
    #   dim 10: [rigid, fluid, root_0, root_1, gripper_0, gripper_1, mass_inv,
    #            clusterStiffness, clusterPlasticThreshold, cluasterPlasticCreep]
    attr = np.zeros((count_nodes, args.attr_dim))
    # no need to include mass for now
    # attr[:, 6] = positions[:, -1].data.cpu().numpy() if var else positions[:, -1] # mass_inv
    if args.env == 'RiceGrip':
        # clusterStiffness, clusterPlasticThreshold, cluasterPlasticCreep
        attr[:, -3:] = scene_params[-3:]


    ### construct relations
    Rr_idxs = []        # relation receiver idx list
    Rs_idxs = []        # relation sender idx list
    Ras = []            # relation attributes list
    values = []         # relation value list (should be 1)
    node_r_idxs = []    # list of corresponding receiver node idx
    node_s_idxs = []    # list of corresponding sender node idx
    psteps = []         # propagation steps

    ##### add env specific graph components
    rels = []
    if args.env == 'RiceGrip':
        # nodes = np.arange(n_particles)
        for i in range(n_shapes):
            attr[n_particles + i, 2 + i] = 1

            pos = positions.data.cpu().numpy() if var else positions
            dis = np.linalg.norm(
                pos[:n_particles, 3:6:2] - pos[n_particles + i, 3:6:2], axis=1)
            nodes = np.nonzero(dis < 0.3)[0]

            if verbose:
                visualize_neighbors(positions, positions, 0, nodes)
                print(np.sort(dis)[:10])

            gripper = np.ones(nodes.shape[0], dtype=np.int) * (n_particles + i)
            rels += [np.stack([nodes, gripper, np.ones(nodes.shape[0])], axis=1)]
            
    elif args.env == 'Hairs':
        # TODO: add relations between the hairs and the stick
        attr[:,0] = 1
        pass

    elif args.env == 'FluidShake':
        for i in range(n_shapes):
            attr[n_particles + i, 1 + i] = 1

            pos = positions.data.cpu().numpy() if var else positions
            if i == 0:
                # floor
                dis = pos[:n_particles, 1] - pos[n_particles + i, 1]
            elif i == 1:
                # left
                dis = pos[:n_particles, 0] - pos[n_particles + i, 0]
            elif i == 2:
                # right
                dis = pos[n_particles + i, 0] - pos[:n_particles, 0]
            elif i == 3:
                # back
                dis = pos[:n_particles, 2] - pos[n_particles + i, 2]
            elif i == 4:
                # front
                dis = pos[n_particles + i, 2] - pos[:n_particles, 2]
            else:
                raise AssertionError("more shapes than expected")
            nodes = np.nonzero(dis < 0.1)[0]

            if verbose:
                visualize_neighbors(positions, positions, 0, nodes)
                print(np.sort(dis)[:10])

            wall = np.ones(nodes.shape[0], dtype=np.int) * (n_particles + i)
            rels += [np.stack([nodes, wall, np.ones(nodes.shape[0])], axis=1)]

    if verbose and len(rels) > 0:
        print(np.concatenate(rels, 0).shape)

    ##### add relations between leaf particles
    for i in range(len(instance_idx) - 1):
        st, ed = instance_idx[i], instance_idx[i + 1]

        if verbose:
            print('instance #%d' % i, st, ed)

        if args.env == 'BoxBath':
            if phases_dict['material'][i] == 'rigid':
                attr[st:ed, 0] = 1
                queries = np.arange(st, ed)
                anchors = np.concatenate((np.arange(st), np.arange(ed, n_particles)))
            elif phases_dict['material'][i] == 'fluid':
                attr[st:ed, 1] = 1
                queries = np.arange(st, ed)
                anchors = np.arange(n_particles)
            else:
                raise AssertionError("Unsupported materials")
                
        elif args.env == 'Hairs':
            # TODO: add relations between the hairs and the stick
            pass
            
          #  if ed not in hairs_idx_begin:
             #   queries = np.arange(st, ed)
              #  anchors = np.arange(n_particles)

        elif args.env == 'FluidFall' or args.env == 'RiceGrip' or args.env == 'FluidShake':
            if phases_dict['material'][i] == 'fluid':
                attr[st:ed, 0] = 1
                queries = np.arange(st, ed)
                anchors = np.arange(n_particles)
            else:
                raise AssertionError("Unsupported materials")

        else:
            raise AssertionError("Unsupported materials")

        # st_time = time.time()
        pos = positions
        pos = pos[:, -3:]
        if args.env == 'Hairs':
            #TODO
            pass
        else:
            rels += find_relations_neighbor(pos, queries, anchors, args.neighbor_radius, 2, var)
            # return list of [receiver, sender, relation_type]
        # print("Time on neighbor search", time.time() - st_time)

    if verbose:
        print("Attr shape (after add env specific graph components):", attr.shape)
        print("Object attr:", np.sum(attr, axis=0))

    rels = np.concatenate(rels, 0)
    if rels.shape[0] > 0:
        if verbose:
            print("Relations neighbor", rels.shape)
        Rr_idxs.append(torch.LongTensor([rels[:, 0], np.arange(rels.shape[0])]))
        Rs_idxs.append(torch.LongTensor([rels[:, 1], np.arange(rels.shape[0])]))
        Ra = np.zeros((rels.shape[0], args.relation_dim))
        Ras.append(torch.FloatTensor(Ra))
        values.append(torch.FloatTensor([1] * rels.shape[0]))
        node_r_idxs.append(np.arange(n_particles))
        node_s_idxs.append(np.arange(n_particles + n_shapes))
        psteps.append(args.pstep)

    if verbose:
        print('clusters', clusters)

    # add heirarchical relations per instance
    cnt_clusters = 0
    for i in range(len(instance_idx) - 1):
        st, ed = instance_idx[i], instance_idx[i + 1]
        n_root_level = len(phases_dict["root_num"][i])

        if n_root_level > 0:
            attr, positions, velocities, count_nodes, \
            rels, node_r_idx, node_s_idx, pstep = \
                    make_hierarchy(args.env, attr, positions, velocities, i, st, ed,
                                   phases_dict, count_nodes, clusters[cnt_clusters], verbose, var)

            for j in range(len(rels)):
                if verbose:
                    print("Relation instance", j, rels[j].shape)
                Rr_idxs.append(torch.LongTensor([rels[j][:, 0], np.arange(rels[j].shape[0])]))
                Rs_idxs.append(torch.LongTensor([rels[j][:, 1], np.arange(rels[j].shape[0])]))
                Ra = np.zeros((rels[j].shape[0], args.relation_dim)); Ra[:, 0] = 1
                Ras.append(torch.FloatTensor(Ra))
                values.append(torch.FloatTensor([1] * rels[j].shape[0]))
                node_r_idxs.append(node_r_idx[j])
                node_s_idxs.append(node_s_idx[j])
                psteps.append(pstep[j])

            cnt_clusters += 1

    if verbose:
        if args.env == 'RiceGrip' or args.env == 'FluidShake':
            print("Scene_params:", scene_params)

        print("Attr shape (after hierarchy building):", attr.shape)
        print("Object attr:", np.sum(attr, axis=0))
        print("Particle attr:", np.sum(attr[:n_particles], axis=0))
        print("Shape attr:", np.sum(attr[n_particles:n_particles+n_shapes], axis=0))
        print("Roots attr:", np.sum(attr[n_particles+n_shapes:], axis=0))

    ### normalize data
    data = [positions, velocities]
    data = normalize(data, stat, var)
    positions, velocities = data[0], data[1]

    if verbose:
        print("Particle positions stats")
        print(positions.shape)
        print(np.min(positions[:n_particles], 0))
        print(np.max(positions[:n_particles], 0))
        print(np.mean(positions[:n_particles], 0))
        print(np.std(positions[:n_particles], 0))

        show_vel_dim = 6 if args.env == 'RiceGrip' else 3
        print("Velocities stats")
        print(velocities.shape)
        print(np.mean(velocities[:n_particles, :show_vel_dim], 0))
        print(np.std(velocities[:n_particles, :show_vel_dim], 0))

    if args.env == 'RiceGrip':
        if var:
            quats = torch.cat(
                [Variable(torch.zeros(n_particles, 4).cuda()), shape_quats,
                 Variable(torch.zeros(count_nodes - n_particles - n_shapes, 4).cuda())], 0)
            state = torch.cat([positions, velocities, quats], 1)
        else:
            quat_null = np.array([[0., 0., 0., 0.]])
            quats = np.repeat(quat_null, [count_nodes], axis=0)
            quats[n_particles:n_particles + n_shapes] = shape_quats
            # if args.eval == 0:
            # quats += np.random.randn(quats.shape[0], 4) * 0.05
            state = torch.FloatTensor(np.concatenate([positions, velocities, quats], axis=1))
    else:
        if var:
            state = torch.cat([positions, velocities], 1)
        else:
            state = torch.FloatTensor(np.concatenate([positions, velocities], axis=1))

    if verbose:
        for i in range(count_nodes - 1):
            if np.sum(np.abs(attr[i] - attr[i + 1])) > 1e-6:
                print(i, attr[i], attr[i + 1])

        for i in range(len(Ras)):
            print(i, np.min(node_r_idxs[i]), np.max(node_r_idxs[i]), np.min(node_s_idxs[i]), np.max(node_s_idxs[i]))

    attr = torch.FloatTensor(attr)
    relations = [Rr_idxs, Rs_idxs, values, Ras, node_r_idxs, node_s_idxs, psteps]

    return attr, state, relations, n_particles, n_shapes, instance_idx

In [ ]:
def gen_PyFleX(info):

    env, root_num = info['env'], info['root_num']
    thread_idx, data_dir, data_names = info['thread_idx'], info['data_dir'], info['data_names']
    n_rollout, n_instance = info['n_rollout'], info['n_instance']
    time_step, time_step_clip = info['time_step'], info['time_step_clip']
    shape_state_dim, dt = info['shape_state_dim'], info['dt']

    env_idx = info['env_idx']

    np.random.seed(round(time.time() * 1000 + thread_idx) % 2**32)
    
    stats = [init_stat(3), init_stat(3)]

    import pyflex
    pyflex.init()

    for i in range(n_rollout):

        if i % 10 == 0:
            print("%d / %d" % (i, n_rollout))

        rollout_idx = thread_idx * n_rollout + i
        rollout_dir = os.path.join(data_dir, str(rollout_idx))
        os.system('mkdir -p ' + rollout_dir)
        
        env_idx = 10
        # scene_params: [len(box) at dim x,len(box) at dim y,len(box) at dim z, num_hair per circle, num_circle]
        box_size = [0.1,0.1,1.5]
        N_p_cirlcle = 50
        N_circle = 20
        N_hairs = N_circle*N_p_cirlcle


        scene_params = np.array(box_size + [N_p_cirlcle] + [N_circle])

        pyflex.set_scene(env_idx, scene_params, thread_idx)
        n_particles = pyflex.get_n_particles()
        N_particles_per_hair = int(n_particles/N_hairs)
        idx_begins = np.arange(N_hairs)*N_particles_per_hair
        idx_hairs = [[i,i+N_particles_per_hair-1] for i in idx_begins]

        positions = np.zeros((time_step, n_particles, 3), dtype=np.float32)
        velocities = np.zeros((time_step, n_particles, 3), dtype=np.float32)

        for j in range(time_step_clip):
            p_clip = pyflex.get_positions().reshape(-1, 4)[:, :3]
            pyflex.step()

        for j in range(time_step):
            positions[j] = pyflex.get_positions().reshape(-1, 4)[:, :3]
            if j == 0:
                velocities[j] = (positions[j] - p_clip) / dt
            else:
                velocities[j] = (positions[j] - positions[j - 1]) / dt

            pyflex.step()
            data = [positions[j], velocities[j], idx_hairs]
            store_data(data_names, data, os.path.join(rollout_dir, str(j) + '.h5'))
        
        # change dtype for more accurate stat calculation
        # only normalize positions and velocities
        datas = [positions.astype(np.float64), velocities.astype(np.float64)]

        for j in range(len(stats)): 
            # here j = 2, refers to positions and velocities
            stat = init_stat(stats[j].shape[0]) 
            # stat= np.zeros((3,3))
            stat[:, 0] = np.mean(datas[j], axis=(0, 1))[:]
            stat[:, 1] = np.std(datas[j], axis=(0, 1))[:]
            stat[:, 2] = datas[j].shape[0] * datas[j].shape[1] # time_step*n_particles
            stats[j] = combine_stat(stats[j], stat)

    pyflex.clean()

    return stats

In [ ]:
class PhysicsFleXDataset(Dataset):

    def __init__(self, args, phase, phases_dict, verbose):
        self.args = args
        self.phase = phase
        self.phases_dict = phases_dict
        self.verbose = verbose
        self.data_dir = os.path.join(self.args.dataf, phase)
        self.stat_path = os.path.join(self.args.dataf, 'stat.h5')

        os.system('mkdir -p ' + self.data_dir)

        #    self.data_names = ['positions', 'velocities', 'shape_quats', 'clusters', 'scene_params']
        self.data_names = ['positions', 'velocities','hair_idx']

        ratio = self.args.train_valid_ratio
        if phase == 'train':
            self.n_rollout = int(self.args.n_rollout * ratio)
        elif phase == 'valid':
            self.n_rollout = self.args.n_rollout - int(self.args.n_rollout * ratio)
        else:
            raise AssertionError("Unknown phase")

    def __len__(self):
        return self.n_rollout * (self.args.time_step - 1)

    def load_data(self, name):
        self.stat = load_data(self.data_names[:2], self.stat_path)
        for i in range(len(self.stat)):
            self.stat[i] = self.stat[i][-self.args.position_dim:, :]
            # print(self.data_names[i], self.stat[i].shape)

    def gen_data(self, name):
        # if the data hasn't been generated, generate the data
        print("Generating data ... n_rollout=%d, time_step=%d" % (self.n_rollout, self.args.time_step))

        infos = []
        for i in range(self.args.num_workers):
            info = {
                'env': self.args.env,
                'root_num': self.phases_dict['root_num'],
                'thread_idx': i,
                'data_dir': self.data_dir,
                'data_names': self.data_names,
                'n_rollout': self.n_rollout // self.args.num_workers,
                'n_instance': self.args.n_instance,
                'time_step': self.args.time_step,
                'time_step_clip': self.args.time_step_clip,
                'dt': self.args.dt,
                'shape_state_dim': self.args.shape_state_dim}

            info['env_idx'] = 10
            infos.append(info)

        cores = self.args.num_workers
        pool = mp.Pool(processes=cores)
        data = pool.map(gen_PyFleX, infos)

        print("Training data generated, warpping up stats ...")

        if self.phase == 'train' and self.args.gen_stat:
            # positions [x, y, z], velocities[xdot, ydot, zdot]
            if self.args.env == 'RiceGrip':
                self.stat = [init_stat(6), init_stat(6)]
            else:
                self.stat = [init_stat(3), init_stat(3)]
            for i in range(len(data)):
                for j in range(len(self.stat)):
                    self.stat[j] = combine_stat(self.stat[j], data[i][j])
            store_data(self.data_names[:2], self.stat, self.stat_path)
        else:
            print("Loading stat from %s ..." % self.stat_path)
            self.stat = load_data(self.data_names[:2], self.stat_path)

    def __getitem__(self, idx):
        idx_rollout = idx // (self.args.time_step - 1)
        idx_timestep = idx % (self.args.time_step - 1)

        # ignore the first frame for env RiceGrip
        if self.args.env == 'RiceGrip' and idx_timestep == 0:
            idx_timestep = np.random.randint(1, self.args.time_step - 1)

        data_path = os.path.join(self.data_dir, str(idx_rollout), str(idx_timestep) + '.h5')
        data_nxt_path = os.path.join(self.data_dir, str(idx_rollout), str(idx_timestep + 1) + '.h5')

        data = load_data(self.data_names, data_path)

        vel_his = []
        for i in range(self.args.n_his):
            path = os.path.join(self.data_dir, str(idx_rollout), str(max(1, idx_timestep - i - 1)) + '.h5')
            data_his = load_data(self.data_names, path)
            vel_his.append(data_his[1])

        data[1] = np.concatenate([data[1]] + vel_his, 1)

        attr, state, relations, n_particles, n_shapes, instance_idx = \
                prepare_input(data, self.stat, self.args, self.phases_dict, self.verbose)

        ### label
        data_nxt = normalize(load_data(self.data_names, data_nxt_path), self.stat)

        label = torch.FloatTensor(data_nxt[1][:n_particles])

        return attr, state, relations, n_particles, n_shapes, instance_idx, label

In [ ]:
def prepare_input(data, stat, args, phases_dict, verbose=0, var=False):

    # Arrangement:
    # particles, shapes, roots

    if args.env == 'RiceGrip':
        positions, velocities, shape_quats, clusters, scene_params = data
        n_shapes = shape_quats.size(0) if var else shape_quats.shape[0]
    elif args.env == 'FluidShake':
        positions, velocities, shape_quats, scene_params = data
        n_shapes = shape_quats.size(0) if var else shape_quats.shape[0]
        clusters = None
    elif args.env == 'BoxBath':
        positions, velocities, clusters = data
        n_shapes = 0
    elif args.env == 'FluidFall':
        positions, velocities = data
        n_shapes = 0
        clusters = None
    elif args.env == 'Hairs':
        positions, velocities, hairs_idx = data
        n_shapes = 1
        hairs_idx_begin = [idx[0] for idx in hairs_idx]
        clusters = None

    count_nodes = positions.size(0) if var else positions.shape[0]
    n_particles = count_nodes - n_shapes

    if verbose:
        print("positions", positions.shape)
        print("velocities", velocities.shape)

        print("n_particles", n_particles)
        print("n_shapes", n_shapes)
        if args.env == 'RiceGrip' or args.env == 'FluidShake':
            print("shape_quats", shape_quats.shape)

    ### instance idx
    #   instance_idx (n_instance + 1): start idx of instance
    if args.env == 'RiceGrip' or args.env == 'FluidShake':
        instance_idx = [0, n_particles]
    elif args.env == 'Hairs':
        instance_idx = [0, count_nodes]
    else:
        instance_idx = phases_dict["instance_idx"]
    if verbose:
        print("Instance_idx:", instance_idx)


    ### object attributes
    #   dim 10: [rigid, fluid, root_0, root_1, gripper_0, gripper_1, mass_inv,
    #            clusterStiffness, clusterPlasticThreshold, cluasterPlasticCreep]
    attr = np.zeros((count_nodes, args.attr_dim))
    # no need to include mass for now
    # attr[:, 6] = positions[:, -1].data.cpu().numpy() if var else positions[:, -1] # mass_inv
    if args.env == 'RiceGrip':
        # clusterStiffness, clusterPlasticThreshold, cluasterPlasticCreep
        attr[:, -3:] = scene_params[-3:]


    ### construct relations
    Rr_idxs = []        # relation receiver idx list
    Rs_idxs = []        # relation sender idx list
    Ras = []            # relation attributes list
    values = []         # relation value list (should be 1)
    node_r_idxs = []    # list of corresponding receiver node idx
    node_s_idxs = []    # list of corresponding sender node idx
    psteps = []         # propagation steps

    ##### add env specific graph components
    rels = []
    if args.env == 'RiceGrip':
        # nodes = np.arange(n_particles)
        for i in range(n_shapes):
            attr[n_particles + i, 2 + i] = 1

            pos = positions.data.cpu().numpy() if var else positions
            dis = np.linalg.norm(
                pos[:n_particles, 3:6:2] - pos[n_particles + i, 3:6:2], axis=1)
            nodes = np.nonzero(dis < 0.3)[0]

            if verbose:
                visualize_neighbors(positions, positions, 0, nodes)
                print(np.sort(dis)[:10])

            gripper = np.ones(nodes.shape[0], dtype=np.int) * (n_particles + i)
            rels += [np.stack([nodes, gripper, np.ones(nodes.shape[0])], axis=1)]
            
    elif args.env == 'Hairs':
        # TODO: add relations between the hairs and the stick
        attr[:,0] = 1
        pass

    elif args.env == 'FluidShake':
        for i in range(n_shapes):
            attr[n_particles + i, 1 + i] = 1

            pos = positions.data.cpu().numpy() if var else positions
            if i == 0:
                # floor
                dis = pos[:n_particles, 1] - pos[n_particles + i, 1]
            elif i == 1:
                # left
                dis = pos[:n_particles, 0] - pos[n_particles + i, 0]
            elif i == 2:
                # right
                dis = pos[n_particles + i, 0] - pos[:n_particles, 0]
            elif i == 3:
                # back
                dis = pos[:n_particles, 2] - pos[n_particles + i, 2]
            elif i == 4:
                # front
                dis = pos[n_particles + i, 2] - pos[:n_particles, 2]
            else:
                raise AssertionError("more shapes than expected")
            nodes = np.nonzero(dis < 0.1)[0]

            if verbose:
                visualize_neighbors(positions, positions, 0, nodes)
                print(np.sort(dis)[:10])

            wall = np.ones(nodes.shape[0], dtype=np.int) * (n_particles + i)
            rels += [np.stack([nodes, wall, np.ones(nodes.shape[0])], axis=1)]

    if verbose and len(rels) > 0:
        print(np.concatenate(rels, 0).shape)

    ##### add relations between leaf particles
    for i in range(len(instance_idx) - 1):
        st, ed = instance_idx[i], instance_idx[i + 1]

        if verbose:
            print('instance #%d' % i, st, ed)

        if args.env == 'BoxBath':
            if phases_dict['material'][i] == 'rigid':
                attr[st:ed, 0] = 1
                queries = np.arange(st, ed)
                anchors = np.concatenate((np.arange(st), np.arange(ed, n_particles)))
            elif phases_dict['material'][i] == 'fluid':
                attr[st:ed, 1] = 1
                queries = np.arange(st, ed)
                anchors = np.arange(n_particles)
            else:
                raise AssertionError("Unsupported materials")
                
        elif args.env == 'Hairs':
            # TODO: add relations between the hairs and the stick
            pass
            
          #  if ed not in hairs_idx_begin:
             #   queries = np.arange(st, ed)
              #  anchors = np.arange(n_particles)

        elif args.env == 'FluidFall' or args.env == 'RiceGrip' or args.env == 'FluidShake':
            if phases_dict['material'][i] == 'fluid':
                attr[st:ed, 0] = 1
                queries = np.arange(st, ed)
                anchors = np.arange(n_particles)
            else:
                raise AssertionError("Unsupported materials")

        else:
            raise AssertionError("Unsupported materials")

        # st_time = time.time()
        pos = positions
        pos = pos[:, -3:]
        if args.env == 'Hairs':
            #TODO
            pass
        else:
            rels += find_relations_neighbor(pos, queries, anchors, args.neighbor_radius, 2, var)
            # return list of [receiver, sender, relation_type]
        # print("Time on neighbor search", time.time() - st_time)

    if verbose:
        print("Attr shape (after add env specific graph components):", attr.shape)
        print("Object attr:", np.sum(attr, axis=0))

    rels = np.concatenate(rels, 0)
    if rels.shape[0] > 0:
        if verbose:
            print("Relations neighbor", rels.shape)
        Rr_idxs.append(torch.LongTensor([rels[:, 0], np.arange(rels.shape[0])]))
        Rs_idxs.append(torch.LongTensor([rels[:, 1], np.arange(rels.shape[0])]))
        Ra = np.zeros((rels.shape[0], args.relation_dim))
        Ras.append(torch.FloatTensor(Ra))
        values.append(torch.FloatTensor([1] * rels.shape[0]))
        node_r_idxs.append(np.arange(n_particles))
        node_s_idxs.append(np.arange(n_particles + n_shapes))
        psteps.append(args.pstep)

    if verbose:
        print('clusters', clusters)

    # add heirarchical relations per instance
    cnt_clusters = 0
    for i in range(len(instance_idx) - 1):
        st, ed = instance_idx[i], instance_idx[i + 1]
        n_root_level = len(phases_dict["root_num"][i])

        if n_root_level > 0:
            attr, positions, velocities, count_nodes, \
            rels, node_r_idx, node_s_idx, pstep = \
                    make_hierarchy(args.env, attr, positions, velocities, i, st, ed,
                                   phases_dict, count_nodes, clusters[cnt_clusters], verbose, var)

            for j in range(len(rels)):
                if verbose:
                    print("Relation instance", j, rels[j].shape)
                Rr_idxs.append(torch.LongTensor([rels[j][:, 0], np.arange(rels[j].shape[0])]))
                Rs_idxs.append(torch.LongTensor([rels[j][:, 1], np.arange(rels[j].shape[0])]))
                Ra = np.zeros((rels[j].shape[0], args.relation_dim)); Ra[:, 0] = 1
                Ras.append(torch.FloatTensor(Ra))
                values.append(torch.FloatTensor([1] * rels[j].shape[0]))
                node_r_idxs.append(node_r_idx[j])
                node_s_idxs.append(node_s_idx[j])
                psteps.append(pstep[j])

            cnt_clusters += 1

    if verbose:
        if args.env == 'RiceGrip' or args.env == 'FluidShake':
            print("Scene_params:", scene_params)

        print("Attr shape (after hierarchy building):", attr.shape)
        print("Object attr:", np.sum(attr, axis=0))
        print("Particle attr:", np.sum(attr[:n_particles], axis=0))
        print("Shape attr:", np.sum(attr[n_particles:n_particles+n_shapes], axis=0))
        print("Roots attr:", np.sum(attr[n_particles+n_shapes:], axis=0))

    ### normalize data
    data = [positions, velocities]
    data = normalize(data, stat, var)
    positions, velocities = data[0], data[1]

    if verbose:
        print("Particle positions stats")
        print(positions.shape)
        print(np.min(positions[:n_particles], 0))
        print(np.max(positions[:n_particles], 0))
        print(np.mean(positions[:n_particles], 0))
        print(np.std(positions[:n_particles], 0))

        show_vel_dim = 6 if args.env == 'RiceGrip' else 3
        print("Velocities stats")
        print(velocities.shape)
        print(np.mean(velocities[:n_particles, :show_vel_dim], 0))
        print(np.std(velocities[:n_particles, :show_vel_dim], 0))

    if args.env == 'RiceGrip':
        if var:
            quats = torch.cat(
                [Variable(torch.zeros(n_particles, 4).cuda()), shape_quats,
                 Variable(torch.zeros(count_nodes - n_particles - n_shapes, 4).cuda())], 0)
            state = torch.cat([positions, velocities, quats], 1)
        else:
            quat_null = np.array([[0., 0., 0., 0.]])
            quats = np.repeat(quat_null, [count_nodes], axis=0)
            quats[n_particles:n_particles + n_shapes] = shape_quats
            # if args.eval == 0:
            # quats += np.random.randn(quats.shape[0], 4) * 0.05
            state = torch.FloatTensor(np.concatenate([positions, velocities, quats], axis=1))
    else:
        if var:
            state = torch.cat([positions, velocities], 1)
        else:
            state = torch.FloatTensor(np.concatenate([positions, velocities], axis=1))

    if verbose:
        for i in range(count_nodes - 1):
            if np.sum(np.abs(attr[i] - attr[i + 1])) > 1e-6:
                print(i, attr[i], attr[i + 1])

        for i in range(len(Ras)):
            print(i, np.min(node_r_idxs[i]), np.max(node_r_idxs[i]), np.min(node_s_idxs[i]), np.max(node_s_idxs[i]))

    attr = torch.FloatTensor(attr)
    relations = [Rr_idxs, Rs_idxs, values, Ras, node_r_idxs, node_s_idxs, psteps]

    return attr, state, relations, n_particles, n_shapes, instance_idx

In [ ]:
phases_dict

In [ ]:
# generate data
datasets = {phase: PhysicsFleXDataset(
    args, phase, phases_dict, args.verbose_data) for phase in ['train', 'valid']}

In [ ]:
datasets['train'].load_data(args.env)

In [ ]:
idx = 0
idx_rollout = idx // (datasets['train'].args.time_step - 1)
idx_timestep = idx % (datasets['train'].args.time_step - 1)

# ignore the first frame for env RiceGrip
if datasets['train'].args.env == 'RiceGrip' and idx_timestep == 0:
    idx_timestep = np.random.randint(1, datasets['train'].args.time_step - 1)

data_path = os.path.join(datasets['train'].data_dir, str(idx_rollout), str(idx_timestep) + '.h5')
data_nxt_path = os.path.join(datasets['train'].data_dir, str(idx_rollout), str(idx_timestep + 1) + '.h5')

data = load_data(datasets['train'].data_names, data_path)

vel_his = []
for i in range(datasets['train'].args.n_his):
    path = os.path.join(datasets['train'].data_dir, str(idx_rollout), str(max(1, idx_timestep - i - 1)) + '.h5')
    data_his = load_data(datasets['train'].data_names, path)
    vel_his.append(data_his[1])

data[1] = np.concatenate([data[1]] + vel_his, 1)


In [ ]:
datasets['train'].stat

In [3]:
data

NameError: name 'data' is not defined

In [ ]:
positions, velocities, hairs_idx = data
n_shapes = 1
hairs_idx_begin = [idx[0] for idx in hairs_idx]
hairs_idx_end = [idx[1] for idx in hairs_idx]

clusters = None

n_particles = positions.size


In [ ]:
positions[:3,0:3]

In [ ]:
instance_idx = [0, n_particles]
### object attributes
count_nodes = n_particles
attr = np.zeros((count_nodes, args.attr_dim))

In [ ]:

##### add env specific graph components
rels = []

In [ ]:
[1]*2

In [ ]:
attr[:n_particles,0] = 1
#rels += find_relations_neighbor(pos, queries, anchors, args.neighbor_radius, 2, var)
#queries = set of hairs, anchors = stick
for i in range(n_particles):
    if i in hairs_idx_begin:
        receiver = [i+1]
        sender = [i]
        relation_type = [1]
    elif i in hairs_idx_end:
        receiver = [i-1]
        sender = [i]
        relation_type = [1]
    else:
        receiver = [i-1,i+1]
        sender = [i, i]
        relation_type = [1,1]
    rels.append(np.stack([receiver, sender, relation_type], axis=1))
    

In [ ]:
rels

In [ ]:
rels = np.concatenate(rels, 0)

In [ ]:
### construct relations
Rr_idxs = []        # relation receiver idx list
Rs_idxs = []        # relation sender idx list
Ras = []            # relation attributes list
values = []         # relation value list (should be 1)
node_r_idxs = []    # list of corresponding receiver node idx
node_s_idxs = []    # list of corresponding sender node idx
psteps = []         # propagation steps


Rr_idxs.append(torch.LongTensor([rels[:, 0], np.arange(rels.shape[0])]))
Rs_idxs.append(torch.LongTensor([rels[:, 1], np.arange(rels.shape[0])]))
Ra = np.zeros((rels.shape[0], args.relation_dim))
Ras.append(torch.FloatTensor(Ra))
values.append(torch.FloatTensor([1] * rels.shape[0]))
node_r_idxs.append(np.arange(n_particles))
node_s_idxs.append(np.arange(n_particles + n_shapes))
psteps.append(args.pstep)

In [ ]:
data = [positions, velocities]
stat = datasets['train'].stat
data = normalize(data, stat)
positions, velocities = data[0], data[1]

In [ ]:
attr = torch.FloatTensor(attr)
relations = [Rr_idxs, Rs_idxs, values, Ras, node_r_idxs, node_s_idxs, psteps]

In [ ]:
relations

In [ ]:
def prepare_input(data, stat, args, phases_dict, verbose=0, var=False):
    
    positions, velocities, hairs_idx = data
    n_shapes = 1
    hairs_idx_begin = [idx[0] for idx in hairs_idx]
    clusters = None

    count_nodes = positions.size(0) if var else positions.shape[0]
    n_particles = count_nodes - n_shapes

    ### instance idx
    #   instance_idx (n_instance + 1): start idx of instance
    if args.env == 'RiceGrip' or args.env == 'FluidShake':
        instance_idx = [0, n_particles]
    elif args.env == 'Hairs':
        instance_idx = [0, count_nodes]
    else:
        instance_idx = phases_dict["instance_idx"]
    if verbose:
        print("Instance_idx:", instance_idx)


    ### object attributes
    #   dim 10: [rigid, fluid, root_0, root_1, gripper_0, gripper_1, mass_inv,
    #            clusterStiffness, clusterPlasticThreshold, cluasterPlasticCreep]
    attr = np.zeros((count_nodes, args.attr_dim))
    # no need to include mass for now
    # attr[:, 6] = positions[:, -1].data.cpu().numpy() if var else positions[:, -1] # mass_inv
    if args.env == 'RiceGrip':
        # clusterStiffness, clusterPlasticThreshold, cluasterPlasticCreep
        attr[:, -3:] = scene_params[-3:]


    ### construct relations
    Rr_idxs = []        # relation receiver idx list
    Rs_idxs = []        # relation sender idx list
    Ras = []            # relation attributes list
    values = []         # relation value list (should be 1)
    node_r_idxs = []    # list of corresponding receiver node idx
    node_s_idxs = []    # list of corresponding sender node idx
    psteps = []         # propagation steps

    ##### add env specific graph components
    rels = []
    if args.env == 'RiceGrip':
        # nodes = np.arange(n_particles)
        for i in range(n_shapes):
            attr[n_particles + i, 2 + i] = 1

            pos = positions.data.cpu().numpy() if var else positions
            dis = np.linalg.norm(
                pos[:n_particles, 3:6:2] - pos[n_particles + i, 3:6:2], axis=1)
            nodes = np.nonzero(dis < 0.3)[0]

            if verbose:
                visualize_neighbors(positions, positions, 0, nodes)
                print(np.sort(dis)[:10])

            gripper = np.ones(nodes.shape[0], dtype=np.int) * (n_particles + i)
            rels += [np.stack([nodes, gripper, np.ones(nodes.shape[0])], axis=1)]
            
    elif args.env = 'Hairs':
        # TODO: add relations between the hairs and the stick
        attr[:,0] = 1
        pass

    elif args.env == 'FluidShake':
        for i in range(n_shapes):
            attr[n_particles + i, 1 + i] = 1

            pos = positions.data.cpu().numpy() if var else positions
            if i == 0:
                # floor
                dis = pos[:n_particles, 1] - pos[n_particles + i, 1]
            elif i == 1:
                # left
                dis = pos[:n_particles, 0] - pos[n_particles + i, 0]
            elif i == 2:
                # right
                dis = pos[n_particles + i, 0] - pos[:n_particles, 0]
            elif i == 3:
                # back
                dis = pos[:n_particles, 2] - pos[n_particles + i, 2]
            elif i == 4:
                # front
                dis = pos[n_particles + i, 2] - pos[:n_particles, 2]
            else:
                raise AssertionError("more shapes than expected")
            nodes = np.nonzero(dis < 0.1)[0]

            if verbose:
                visualize_neighbors(positions, positions, 0, nodes)
                print(np.sort(dis)[:10])

            wall = np.ones(nodes.shape[0], dtype=np.int) * (n_particles + i)
            rels += [np.stack([nodes, wall, np.ones(nodes.shape[0])], axis=1)]

    if verbose and len(rels) > 0:
        print(np.concatenate(rels, 0).shape)

    ##### add relations between leaf particles
    for i in range(len(instance_idx) - 1):
        st, ed = instance_idx[i], instance_idx[i + 1]

        if verbose:
            print('instance #%d' % i, st, ed)

        if args.env == 'BoxBath':
            if phases_dict['material'][i] == 'rigid':
                attr[st:ed, 0] = 1
                queries = np.arange(st, ed)
                anchors = np.concatenate((np.arange(st), np.arange(ed, n_particles)))
            elif phases_dict['material'][i] == 'fluid':
                attr[st:ed, 1] = 1
                queries = np.arange(st, ed)
                anchors = np.arange(n_particles)
            else:
                raise AssertionError("Unsupported materials")
                
        elif args.env == 'Hairs':
            # TODO: add relations between the hairs and the stick
            pass
            
          #  if ed not in hairs_idx_begin:
             #   queries = np.arange(st, ed)
              #  anchors = np.arange(n_particles)

        elif args.env == 'FluidFall' or args.env == 'RiceGrip' or args.env == 'FluidShake':
            if phases_dict['material'][i] == 'fluid':
                attr[st:ed, 0] = 1
                queries = np.arange(st, ed)
                anchors = np.arange(n_particles)
            else:
                raise AssertionError("Unsupported materials")

        else:
            raise AssertionError("Unsupported materials")

        # st_time = time.time()
        pos = positions
        pos = pos[:, -3:]
        if args.env == 'Hairs':
            #TODO
            pass
        else:
            rels += find_relations_neighbor(pos, queries, anchors, args.neighbor_radius, 2, var)
            # return list of [receiver, sender, relation_type]
        # print("Time on neighbor search", time.time() - st_time)

    if verbose:
        print("Attr shape (after add env specific graph components):", attr.shape)
        print("Object attr:", np.sum(attr, axis=0))

    rels = np.concatenate(rels, 0)
    if rels.shape[0] > 0:
        if verbose:
            print("Relations neighbor", rels.shape)
        Rr_idxs.append(torch.LongTensor([rels[:, 0], np.arange(rels.shape[0])]))
        Rs_idxs.append(torch.LongTensor([rels[:, 1], np.arange(rels.shape[0])]))
        Ra = np.zeros((rels.shape[0], args.relation_dim))
        Ras.append(torch.FloatTensor(Ra))
        values.append(torch.FloatTensor([1] * rels.shape[0]))
        node_r_idxs.append(np.arange(n_particles))
        node_s_idxs.append(np.arange(n_particles + n_shapes))
        psteps.append(args.pstep)

    if verbose:
        print('clusters', clusters)

    # add heirarchical relations per instance
    cnt_clusters = 0
    for i in range(len(instance_idx) - 1):
        st, ed = instance_idx[i], instance_idx[i + 1]
        n_root_level = len(phases_dict["root_num"][i])

        if n_root_level > 0:
            attr, positions, velocities, count_nodes, \
            rels, node_r_idx, node_s_idx, pstep = \
                    make_hierarchy(args.env, attr, positions, velocities, i, st, ed,
                                   phases_dict, count_nodes, clusters[cnt_clusters], verbose, var)

            for j in range(len(rels)):
                if verbose:
                    print("Relation instance", j, rels[j].shape)
                Rr_idxs.append(torch.LongTensor([rels[j][:, 0], np.arange(rels[j].shape[0])]))
                Rs_idxs.append(torch.LongTensor([rels[j][:, 1], np.arange(rels[j].shape[0])]))
                Ra = np.zeros((rels[j].shape[0], args.relation_dim)); Ra[:, 0] = 1
                Ras.append(torch.FloatTensor(Ra))
                values.append(torch.FloatTensor([1] * rels[j].shape[0]))
                node_r_idxs.append(node_r_idx[j])
                node_s_idxs.append(node_s_idx[j])
                psteps.append(pstep[j])

            cnt_clusters += 1

    if verbose:
        if args.env == 'RiceGrip' or args.env == 'FluidShake':
            print("Scene_params:", scene_params)

        print("Attr shape (after hierarchy building):", attr.shape)
        print("Object attr:", np.sum(attr, axis=0))
        print("Particle attr:", np.sum(attr[:n_particles], axis=0))
        print("Shape attr:", np.sum(attr[n_particles:n_particles+n_shapes], axis=0))
        print("Roots attr:", np.sum(attr[n_particles+n_shapes:], axis=0))

    ### normalize data
    data = [positions, velocities]
    data = normalize(data, stat, var)
    positions, velocities = data[0], data[1]

    if verbose:
        print("Particle positions stats")
        print(positions.shape)
        print(np.min(positions[:n_particles], 0))
        print(np.max(positions[:n_particles], 0))
        print(np.mean(positions[:n_particles], 0))
        print(np.std(positions[:n_particles], 0))

        show_vel_dim = 6 if args.env == 'RiceGrip' else 3
        print("Velocities stats")
        print(velocities.shape)
        print(np.mean(velocities[:n_particles, :show_vel_dim], 0))
        print(np.std(velocities[:n_particles, :show_vel_dim], 0))

    if args.env == 'RiceGrip':
        if var:
            quats = torch.cat(
                [Variable(torch.zeros(n_particles, 4).cuda()), shape_quats,
                 Variable(torch.zeros(count_nodes - n_particles - n_shapes, 4).cuda())], 0)
            state = torch.cat([positions, velocities, quats], 1)
        else:
            quat_null = np.array([[0., 0., 0., 0.]])
            quats = np.repeat(quat_null, [count_nodes], axis=0)
            quats[n_particles:n_particles + n_shapes] = shape_quats
            # if args.eval == 0:
            # quats += np.random.randn(quats.shape[0], 4) * 0.05
            state = torch.FloatTensor(np.concatenate([positions, velocities, quats], axis=1))
    else:
        if var:
            state = torch.cat([positions, velocities], 1)
        else:
            state = torch.FloatTensor(np.concatenate([positions, velocities], axis=1))

    if verbose:
        for i in range(count_nodes - 1):
            if np.sum(np.abs(attr[i] - attr[i + 1])) > 1e-6:
                print(i, attr[i], attr[i + 1])

        for i in range(len(Ras)):
            print(i, np.min(node_r_idxs[i]), np.max(node_r_idxs[i]), np.min(node_s_idxs[i]), np.max(node_s_idxs[i]))

    attr = torch.FloatTensor(attr)
    relations = [Rr_idxs, Rs_idxs, values, Ras, node_r_idxs, node_s_idxs, psteps]

    return attr, state, relations, n_particles, n_shapes, instance_idx

In [ ]:

attr, state, relations, n_particles, n_shapes, instance_idx = \
        prepare_input(data, self.stat, self.args, self.phases_dict, self.verbose)

### label
data_nxt = normalize(load_data(self.data_names, data_nxt_path), self.stat)

label = torch.FloatTensor(data_nxt[1][:n_particles])

In [ ]:
data = datasets['train'].__getitem__(1)

In [ ]:
print(data)